In [1]:
!pip install gym

In [2]:
from IPython.core.debugger import set_trace
import numpy as np
import pprint
import gym
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [3]:
pp=pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [4]:

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
        
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    
    while True:
        delta=0
        for i in range(env.nS):
            value=0
            #look at the possible actions
            for action,action_prob in enumerate(policy[i]):
                #for each actions look at the next state
                for prob,next_state,reward,done in env.P[i][action]:
                    value+= action_prob*prob*(reward+discount_factor*V[next_state])
            
            delta=max(delta,np.abs(V[i]-value))
            V[i]=value
        if delta<theta:
            break
        
    return np.array(V)


In [5]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [6]:
print("Value Function:")
print(v)

Value Function:
[  0.         -13.99993529 -19.99990698 -21.99989761 -13.99993529
 -17.9999206  -19.99991379 -19.99991477 -19.99990698 -19.99991379
 -17.99992725 -13.99994569 -21.99989761 -19.99991477 -13.99994569
   0.        ]


In [7]:
env.shape

[4, 4]

In [8]:
print("Reshaped Grid Value Function")
print(v.reshape(env.shape))

Reshaped Grid Value Function
[[  0.         -13.99993529 -19.99990698 -21.99989761]
 [-13.99993529 -17.9999206  -19.99991379 -19.99991477]
 [-19.99990698 -19.99991379 -17.99992725 -13.99994569]
 [-21.99989761 -19.99991477 -13.99994569   0.        ]]


In [9]:
random_policy = np.ones([env.nS,env.nA])/env.nA
for a in random_policy[1]:
    print(a)

0.25
0.25
0.25
0.25


In [10]:
random_policy.size

64

In [11]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)